# FASE 1 - DATA STORAGE 

## IMPORT DATASETS

In [132]:
import pandas as pd
import json
import numpy as np 
import pycountry as pc

In [133]:
#DATASETS ESCOLHIDOS

dfLabor = pd.read_csv('datasets/world_labor_productivity.csv', delimiter=',')
dfSalary = pd.read_csv('datasets/world_annual_wage.csv', delimiter=',')
dfMental=pd.read_csv('datasets/mental_illness.csv', delimiter=',')

In [134]:
dfLabor

,Entity,Code,Year,Productivity: output per hour worked
0,Argentina,ARG,1972,5.770799
1,Argentina,ARG,1973,6.049992
2,Argentina,ARG,1974,6.123540
3,Argentina,ARG,1975,6.015602
4,Argentina,ARG,1976,6.029272
...,...,...,...,...
3452,Vietnam,VNM,2015,4.946606
3453,Vietnam,VNM,2016,5.156925
3454,Vietnam,VNM,2017,5.652919
3455,Vietnam,VNM,2018,5.982665


In [135]:
dfMental

,Entity,Code,Year,"DALYs from depressive disorders per 100,000 people in, both sexes aged age-standardized","DALYs from schizophrenia per 100,000 people in, both sexes aged age-standardized","DALYs from bipolar disorder per 100,000 people in, both sexes aged age-standardized","DALYs from eating disorders per 100,000 people in, both sexes aged age-standardized","DALYs from anxiety disorders per 100,000 people in, both sexes aged age-standardized"
0,Afghanistan,AFG,1990,895.22565,138.248250,147.64412,26.471115,440.33000
1,Afghanistan,AFG,1991,893.88434,137.761220,147.56696,25.548681,439.47202
2,Afghanistan,AFG,1992,892.34973,137.080300,147.13086,24.637949,437.60718
3,Afghanistan,AFG,1993,891.51587,136.486020,146.78812,23.863169,436.69104
4,Afghanistan,AFG,1994,891.39160,136.183230,146.58481,23.189074,436.76800
...,...,...,...,...,...,...,...,...
6835,Zimbabwe,ZWE,2015,546.46204,127.108720,115.32073,20.423056,302.20868
6836,Zimbabwe,ZWE,2016,547.27765,127.142105,114.98700,20.647228,302.68216
6837,Zimbabwe,ZWE,2017,547.62270,127.465050,115.32798,20.791725,302.88626
6838,Zimbabwe,ZWE,2018,546.57184,127.681210,115.42796,20.916480,301.58250


In [136]:
dfSalary

,COUNTRY,Country,SERIES,Series,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,CPNCU,Current prices in NCU,2000,2000,AUD,Australian Dollar,0,Units,NaN,NaN,46246.868731,NaN,NaN
1,AUS,Australia,CPNCU,Current prices in NCU,2001,2001,AUD,Australian Dollar,0,Units,NaN,NaN,48315.982391,NaN,NaN
2,AUS,Australia,CPNCU,Current prices in NCU,2002,2002,AUD,Australian Dollar,0,Units,NaN,NaN,50052.758102,NaN,NaN
3,AUS,Australia,CPNCU,Current prices in NCU,2003,2003,AUD,Australian Dollar,0,Units,NaN,NaN,51798.586644,NaN,NaN
4,AUS,Australia,CPNCU,Current prices in NCU,2004,2004,AUD,Australian Dollar,0,Units,NaN,NaN,54199.402711,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,CRI,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2017,2017,NaN,NaN,0,Units,NaN,NaN,27020.000000,NaN,NaN
2605,CRI,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2018,2018,NaN,NaN,0,Units,NaN,NaN,27709.000000,NaN,NaN
2606,CRI,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2019,2019,NaN,NaN,0,Units,NaN,NaN,28524.000000,NaN,NaN
2607,CRI,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2020,2020,NaN,NaN,0,Units,NaN,NaN,31341.000000,NaN,NaN


## TRATAMENTO

In [137]:
print(dfSalary.columns)


Index(['COUNTRY', 'Country', 'SERIES', 'Series', 'TIME', 'Time', 'Unit Code',
       'Unit', 'PowerCode Code', 'PowerCode', 'Reference Period Code',
       'Reference Period', 'Value', 'Flag Codes', 'Flags'],
      dtype='object')


In [138]:
dfSalary.rename(columns={'Country':'Entity'}, inplace=True)
dfSalary.drop(columns=[ 'COUNTRY' ], inplace=True)

In [139]:
dfSalary.rename(columns={'Time':'Year'}, inplace=True)
dfSalary.drop(columns=[ 'TIME' ], inplace=True)

In [140]:
dfSalary

,Entity,SERIES,Series,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,Australia,CPNCU,Current prices in NCU,2000,AUD,Australian Dollar,0,Units,NaN,NaN,46246.868731,NaN,NaN
1,Australia,CPNCU,Current prices in NCU,2001,AUD,Australian Dollar,0,Units,NaN,NaN,48315.982391,NaN,NaN
2,Australia,CPNCU,Current prices in NCU,2002,AUD,Australian Dollar,0,Units,NaN,NaN,50052.758102,NaN,NaN
3,Australia,CPNCU,Current prices in NCU,2003,AUD,Australian Dollar,0,Units,NaN,NaN,51798.586644,NaN,NaN
4,Australia,CPNCU,Current prices in NCU,2004,AUD,Australian Dollar,0,Units,NaN,NaN,54199.402711,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2017,NaN,NaN,0,Units,NaN,NaN,27020.000000,NaN,NaN
2605,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2018,NaN,NaN,0,Units,NaN,NaN,27709.000000,NaN,NaN
2606,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2019,NaN,NaN,0,Units,NaN,NaN,28524.000000,NaN,NaN
2607,Costa Rica,USDPPP,In 2022 constant prices at 2022 USD PPPs,2020,NaN,NaN,0,Units,NaN,NaN,31341.000000,NaN,NaN
